## **In this notebook all the combinations of models, datasets, preprocessing methods done until now will be joined together. The focus will be on Scalability, dinamicity and correct results saving.**







USEFUL LINKS:

https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel

https://towardsdatascience.com/a-novel-approach-to-feature-importance-shapley-additive-explanations-d18af30fc21b

https://shap.readthedocs.io/en/latest/index.html

### Imports section: 


In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/Drive')

Mounted at /content/Drive


In [2]:
# Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score, balanced_accuracy_score, f1_score, matthews_corrcoef, classification_report, make_scorer
from sklearn.linear_model import LogisticRegression, Lasso
import matplotlib.pyplot as plt
from xlwt import Workbook
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import os
from pandas_profiling import ProfileReport
from sklearn import svm
from sklearn.svm import SVC
from datetime import datetime
from sklearn.feature_selection import SelectFromModel

In [3]:
import warnings
warnings.filterwarnings('ignore')
!pip freeze
# ! pip install scikit-learn==0.24.2 # Downgrading the scikit learn library to obtain same results of previous experiments and Convergence

absl-py==1.2.0
aeppl==0.0.33
aesara==2.7.9
aiohttp==3.8.1
aiosignal==1.2.0
alabaster==0.7.12
albumentations==1.2.1
altair==4.2.0
appdirs==1.4.4
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
async-timeout==4.0.2
asynctest==0.13.0
atari-py==0.2.9
atomicwrites==1.4.1
attrs==22.1.0
audioread==3.0.0
autograd==1.4
Babel==2.10.3
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.1
blis==0.7.8
bokeh==2.3.3
branca==0.5.0
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==4.2.4
catalogue==2.0.8
certifi==2022.6.15
cffi==1.15.1
cftime==1.6.1
chardet==3.0.4
charset-normalizer==2.1.1
click==7.1.2
clikit==0.6.2
cloudpickle==1.5.0
cmake==3.22.6
cmdstanpy==1.0.7
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
cons==0.4.5
contextlib2==0.5.5
convertdate==2.4.0
crashtest==0.3.1
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.3.0
cvxpy==1.2.1
cycler==0.11.0
cymem==2.0.6
Cython==0.29.32
daft==0.0.4
dask==2022.2.0
datascience==0.17.5
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.

### Download of all datasets with different preprocessing strategies and feature spaces

In [4]:
# Current working directory and other paths
cwd = os.getcwd()
print(cwd)
!cd Drive/
path = cwd + "/Drive/My Drive/magistrale/BioinformaticsProject/data/"
results_path = cwd + "/Drive/My Drive/magistrale/BioinformaticsProject/results/"

# Count per Million matrix
cpm_dataset = pd.read_csv(path+"CPM.csv",index_col=0) #read the main CPM dataset(67k × 719)
cpm_dataset = cpm_dataset.transpose() # (719 × 67k)
# Training and Testing datasets
training_ds =  pd.read_excel(path+"train.test.xlsx", sheet_name="train")
testing_ds = pd.read_excel(path+"train.test.xlsx", sheet_name="test")

# Feature space datesets
base_feature_space =path+"FEATURE_SPACES(RAW +CPM).xlsx"
# List of feature space name 
feature_space_files =["FEATURE_SPACE6(MAIN)", "FEATURE_SPACE1(PAM)", "FEATURE_SPACE2(PAM)","FEATURE_SPACE1(LIMMA)","FEATURE_SPACE2(LIMMA)", "FEATURE_SPACE7(pamsimilarity)","FEATURE_SPACE8(limmasimilarity)"]

/content


In [5]:
def extract_and_reduce_by_columns(path, sheet_name, name, nofeats_ds, preproc_strategy: str= "none"): 
  """
     Function to extract dataset and a specific group of its columns.

     path: the path where to get the data values (isoforms)
     sheet_name: the excel sheet were to get the columns to select for the data (isoforms)
     nofeats_ds: the dataset without the additional columns
     name: 'trainingset' or 'testingset' for the excel 
     preproc_strategy: which preprocessing strategy to apply to the ds

  """
  full_df = pd.read_excel(path, sheet_name=sheet_name) # path of subdatset 
  full_list= full_df['isoform'].values.tolist()  #exatrct the list of isoforms names as list
  if preproc_strategy == 'loge':
    # https://stackoverflow.com/questions/49538185/purpose-of-numpy-log1p
    log_cpm_dataset = np.log1p(cpm_dataset)
    data = log_cpm_dataset[np.intersect1d(log_cpm_dataset.columns, full_list)]
  elif preproc_strategy == 'log2':
    log_cpm_dataset = np.log2(cpm_dataset + 1) # constant added to avoid reaching zero
    data = log_cpm_dataset[np.intersect1d(log_cpm_dataset.columns, full_list)]
  elif preproc_strategy == 'normperrow':
    # normalize per rows
    data = cpm_dataset[np.intersect1d(cpm_dataset.columns, full_list)]
    data = data.div(data.sum(axis=1), axis=0) # ----> preprocessing scaling step to try, not working
  elif preproc_strategy == 'none':
    data = cpm_dataset[np.intersect1d(cpm_dataset.columns, full_list)]  # find the mutual isoform between main datset and subdatset 
  
  data.reset_index(inplace=True)
  data.rename(columns={ data.columns[0]: "sample_id" }, inplace = True)

  x = nofeats_ds['sample_id'].values.tolist()
  data1= data.loc[data['sample_id'].isin(x)]
  result = pd.merge(data1, nofeats_ds, on='sample_id')
  result
  result.rename(columns={'sample_id.1':'subtype'}, inplace=True )
 
  # result.to_csv(name +".csv", index=False) # save as csv file 
  return result

 DATASET 1 CON FS PAM50
 

In [6]:
# List of feature space name 
feature_space_files =["FEATURE_SPACE6(MAIN)", "FEATURE_SPACE1(PAM)", "FEATURE_SPACE2(PAM)","FEATURE_SPACE1(LIMMA)","FEATURE_SPACE2(LIMMA)", "FEATURE_SPACE7(pamsimilarity)","FEATURE_SPACE8(limmasimilarity)"]

In [7]:
# Training Data import:
train = extract_and_reduce_by_columns(base_feature_space, "FEATURE_SPACE7(pamsimilarity)", 'trainingset', training_ds, 'none') 
X_train_pam = train.drop(["sample_id","subtype"],  axis = 1)
Y_train_pam =train.subtype
print("X_train size:", X_train_pam.shape)

# Testing Data import:
test = extract_and_reduce_by_columns(base_feature_space, "FEATURE_SPACE7(pamsimilarity)",'testingset', testing_ds,'none' ) 
X_test_pam = test.drop(["sample_id","subtype"], axis = 1)
Y_test_pam = test.subtype
print("X_test size:", X_test_pam.shape)

X_train size: (550, 131)
X_test size: (137, 131)


DATASET 2 CON FS LIMMA50

In [8]:
# Training Data import:
train = extract_and_reduce_by_columns(base_feature_space, "FEATURE_SPACE8(limmasimilarity)", 'trainingset', training_ds, 'none' ) 
X_train_limma = train.drop(["sample_id","subtype"],  axis = 1)
Y_train_limma =train.subtype
print("X_train size:", X_train_limma.shape)

# Testing Data import:
test = extract_and_reduce_by_columns(base_feature_space, "FEATURE_SPACE8(limmasimilarity)",'testingset', testing_ds, 'none') 
X_test_limma = test.drop(["sample_id","subtype"], axis = 1)
Y_test_limma = test.subtype
print("X_test size:", X_test_limma.shape)

X_train size: (550, 557)
X_test size: (137, 557)


DATASET 3 CON FS PAM50 E LOGE PREPROC




In [9]:
# Training Data import:
train = extract_and_reduce_by_columns(base_feature_space, "FEATURE_SPACE7(pamsimilarity)", 'trainingset', training_ds, 'loge') 
X_train_pam_loge = train.drop(["sample_id","subtype"],  axis = 1)
Y_train_pam_loge=train.subtype
print("X_train size:", X_train_pam_loge.shape)

# Testing Data import:
test = extract_and_reduce_by_columns(base_feature_space, "FEATURE_SPACE7(pamsimilarity)",'testingset', testing_ds,'loge' ) 
X_test_pam_loge = test.drop(["sample_id","subtype"], axis = 1)
Y_test_pam_loge = test.subtype
print("X_test size:", X_test_pam_loge.shape)

X_train size: (550, 131)
X_test size: (137, 131)


DATASET 4 CON FS LIMMA50 E LOGE PREPROC

In [10]:
# Training Data import:
train = extract_and_reduce_by_columns(base_feature_space, "FEATURE_SPACE8(limmasimilarity)", 'trainingset', training_ds, 'loge' ) 
X_train_limma_loge = train.drop(["sample_id","subtype"],  axis = 1)
Y_train_limma_loge =train.subtype
print("X_train size:", X_train_limma_loge.shape)

# Testing Data import:
test = extract_and_reduce_by_columns(base_feature_space, "FEATURE_SPACE8(limmasimilarity)",'testingset', testing_ds, 'loge') 
X_test_limma_loge = test.drop(["sample_id","subtype"], axis = 1)
Y_test_limma_loge = test.subtype
print("X_test size:", X_test_limma_loge.shape)

X_train size: (550, 557)
X_test size: (137, 557)


DATASET 5 CON FS PAM50 E LOG2 PREPROC

In [11]:
# Training Data import:
train = extract_and_reduce_by_columns(base_feature_space, "FEATURE_SPACE7(pamsimilarity)", 'trainingset', training_ds, 'log2') 
X_train_pam_log2 = train.drop(["sample_id","subtype"],  axis = 1)
Y_train_pam_log2 =train.subtype
print("X_train size:", X_train_pam_log2.shape)

# Testing Data import:
test = extract_and_reduce_by_columns(base_feature_space, "FEATURE_SPACE7(pamsimilarity)",'testingset', testing_ds,'log2' ) 
X_test_pam_log2 = test.drop(["sample_id","subtype"], axis = 1)
Y_test_pam_log2 = test.subtype
print("X_test size:", X_test_pam_log2.shape)

X_train size: (550, 131)
X_test size: (137, 131)


DATASET 6 CON FS LIMMA50 E LOG2 PREPROC

In [12]:
# Training Data import:
train = extract_and_reduce_by_columns(base_feature_space, "FEATURE_SPACE8(limmasimilarity)", 'trainingset', training_ds, 'log2' ) 
X_train_limma_log2 = train.drop(["sample_id","subtype"],  axis = 1)
Y_train_limma_log2 =train.subtype
print("X_train size:", X_train_limma_log2.shape)

# Testing Data import:
test = extract_and_reduce_by_columns(base_feature_space, "FEATURE_SPACE8(limmasimilarity)",'testingset', testing_ds, 'log2') 
X_test_limma_log2 = test.drop(["sample_id","subtype"], axis = 1)
Y_test_limma_log2 = test.subtype
print("X_test size:", X_test_limma_log2.shape)

X_train size: (550, 557)
X_test size: (137, 557)


DATASET 7 CON FS REDUCED FROM PAM50 E LOG PREPROCESSING (choose the best between 2 and e)

In [13]:
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
sel_.fit(X_train_pam_loge,Y_train_pam_loge)

SelectFromModel(estimator=LogisticRegression(C=1, penalty='l1',
                                             solver='liblinear'))

In [14]:
sel_.get_support()
selected_feat = X_train_pam_loge.columns[(sel_.get_support())]

In [15]:
print('total features: {}'.format((X_train_pam_loge.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('Percentage features with coefficients shrank to zero: {}'.format(np.sum(sel_.estimator_.coef_ == 0)/131*5))

total features: 131
selected features: 118
Percentage features with coefficients shrank to zero: 13.931297709923665


In [16]:
X_train_pam_loge_sel = X_train_pam_loge[selected_feat].copy()
Y_train_pam_loge_sel = Y_train_pam_loge

X_test_pam_loge_sel = X_test_pam_loge[selected_feat].copy()
Y_test_pam_loge_sel = Y_test_pam_loge

DATASET 8 CON FS REDUCED FROM LIMMA50 E LOG PREPROCESSING (choose the best between 2 and e)

In [17]:
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
sel_.fit(X_train_limma_loge,Y_train_limma_loge)

SelectFromModel(estimator=LogisticRegression(C=1, penalty='l1',
                                             solver='liblinear'))

In [18]:
sel_.get_support()
selected_feat = X_train_limma.columns[(sel_.get_support())]

In [19]:
print('total features: {}'.format((X_train_limma_loge.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('Percentage features with coefficients shrank to zero: {}'.format(np.sum(sel_.estimator_.coef_ == 0)/557*5))

total features: 557
selected features: 253
Percentage features with coefficients shrank to zero: 21.409335727109514


In [20]:
X_train_limma_loge_sel = X_train_limma_loge[selected_feat].copy()
Y_train_limma_loge_sel = Y_train_limma_loge

X_test_limma_loge_sel = X_test_limma_loge[selected_feat].copy()
Y_test_limma_loge_sel = Y_test_limma_loge

DATASET 9 CON NEW FS created with lasso selection E LOG PREPROCESSING (choose best try e for now) 



####Feature selection is done on a feature space which has been filtered only with some basic noise-removal steps (from 67k to 49k)

In [32]:
# Training Data import:
train = extract_and_reduce_by_columns(base_feature_space, "FEATURE_SPACE6(MAIN)", 'trainingset', training_ds, 'loge') 
X_train_lasso = train.drop(["sample_id","subtype"],  axis = 1)
samples_train = train.sample_id
Y_train_lasso=train.subtype
print("X_train size:", X_train_lasso.shape)

# Testing Data import:
test = extract_and_reduce_by_columns(base_feature_space,"FEATURE_SPACE6(MAIN)", 'testingset', testing_ds,  'loge') 
X_test_lasso = test.drop(["sample_id","subtype"], axis = 1)
Y_test_lasso = test.subtype
print("X_test size:", X_test_lasso.shape)

X_train size: (550, 49740)
X_test size: (137, 49740)


In [33]:
sel_ = SelectFromModel(LogisticRegression(random_state=0, C=1, penalty='l1', solver='liblinear')) # change slightly at every run -> the seed could be fixed for reproducibility
sel_.fit(X_train_lasso,Y_train_lasso)

SelectFromModel(estimator=LogisticRegression(C=1, penalty='l1', random_state=0,
                                             solver='liblinear'))

In [34]:
sel_.get_support()
selected_feat = X_train_lasso.columns[(sel_.get_support())]
print(sel_.estimator_.coef_)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [35]:
print('total features: {}'.format((X_train_lasso.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

perc_feat_selected = np.sum(sel_.estimator_.coef_ == 0)/ (49740*5)
print(perc_feat_selected)

total features: 49740
selected features: 1748
coefficients shrank to zero: 246534
0.9912907117008444


In [36]:
X_train_lasso = X_train_lasso[selected_feat].copy()
Y_train_lasso = Y_train_lasso

X_test_lasso = X_test_lasso[selected_feat].copy()
Y_test_lasso = Y_test_lasso

*New feature space exploration and comparison with fs filtered with pam50 and limma50:*

In [37]:
# Now check how many of the selected features is in either pam50 or limma50 feature space

# pam50 features
pam50_feats = X_train_pam.columns
print("Total number of pam50 features: ",len(pam50_feats))
print("Total number of selected features with lasso: ",len(selected_feat))
i = 0
for p in pam50_feats:
  if p in selected_feat:
    i+=1

print("Number of features from pam50 feature space which were selected with lasso selection: ", i)

# limma50 features
limma50_feats = X_train_limma.columns
print("Total number of pam50 features: ",len(limma50_feats))
print("Total number of selected features with lasso: ",len(selected_feat))
i = 0
for p in limma50_feats:
  if p in selected_feat:
    i+=1

print("Number of features from pam50 feature space which were selected with lasso selection: ", i)

Total number of pam50 features:  131
Total number of selected features with lasso:  1748
Number of features from pam50 feature space which were selected with lasso selection:  62
Total number of pam50 features:  557
Total number of selected features with lasso:  1748
Number of features from pam50 feature space which were selected with lasso selection:  169


DATASET 10 CON NEW FS created with rfclassifier selection E LOG PREPROCESSING (choose best try e for now) 



####Feature selection is done on a feature space which has been filtered only with some basic noise-removal steps (from 67k to 49k)

In [48]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Training Data import:
train = extract_and_reduce_by_columns(base_feature_space, "FEATURE_SPACE6(MAIN)", 'trainingset', training_ds, 'loge') 
X_train_rf = train.drop(["sample_id","subtype"],  axis = 1)
samples_train = train.sample_id
Y_train_rf=train.subtype
print("X_train size:", X_train_rf.shape)

# Testing Data import:
test = extract_and_reduce_by_columns(base_feature_space,"FEATURE_SPACE6(MAIN)", 'testingset', testing_ds,  'loge') 
X_test_rf = test.drop(["sample_id","subtype"], axis = 1)
Y_test_rf = test.subtype
print("X_test size:", X_test_rf.shape)

In [ ]:
sel_ = SelectFromModel(RandomForestClassifier(random_state=0)) # change slightly at every run -> the seed fixed for reproducibility
sel_.fit(X_train_rf,Y_train_rf)

In [ ]:
sel_.get_support()
selected_feat = X_train_rf.columns[(sel_.get_support())]
# print(sel_.estimator_.coef_)

In [ ]:
print('total features: {}'.format((X_train_lasso.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
# random forest classifier selection has no estimator coefficients -> different method for selection of features
#print('coefficients shrank to zero: {}'.format(
#      np.sum(sel_.estimator_.coef_ == 0)))

# perc_feat_selected = np.sum(sel_.estimator_.coef_ == 0)/ (49740*5)
# print(perc_feat_selected)

In [ ]:
X_train_rf = X_train_rf[selected_feat].copy()
Y_train_rf = Y_train_rf

X_test_rf = X_test_rf[selected_feat].copy()
Y_test_rf = Y_test_rf

*New feature space exploration and comparison with fs filtered with pam50 and limma50:*

In [61]:
# Now check how many of the selected features is in either pam50 or limma50 feature space

# pam50 features
pam50_feats = X_train_pam.columns
print("Total number of pam50 features: ",len(pam50_feats))
print("Total number of selected features with lasso: ",len(selected_feat))
i = 0
for p in pam50_feats:
  if p in selected_feat:
    i+=1

print("Number of features from pam50 feature space which were selected with lasso selection: ", i)

# limma50 features
limma50_feats = X_train_limma.columns
print("Total number of pam50 features: ",len(limma50_feats))
print("Total number of selected features with lasso: ",len(selected_feat))
i = 0
for p in limma50_feats:
  if p in selected_feat:
    i+=1

print("Number of features from pam50 feature space which were selected with lasso selection: ", i)

Total number of pam50 features:  131
Total number of selected features with lasso:  4361
Number of features from pam50 feature space which were selected with lasso selection:  52
Total number of pam50 features:  557
Total number of selected features with lasso:  4361
Number of features from pam50 feature space which were selected with lasso selection:  216


### Shapley explanation for Feature importace

In [42]:
LogReg_trained = LogisticRegression(random_state=0, C=0.1,l1_ratio=0.001, multi_class = 'ovr', penalty= 'elasticnet',solver='saga', max_iter=2000).fit(X_train_lasso, Y_train_lasso)

y_pred=LogReg_trained.predict(X_test_lasso)
print("Balanced accuracy: ", round(balanced_accuracy_score(Y_test_lasso, y_pred), 3))
print("Accuracy: ", round(accuracy_score(Y_test_lasso, y_pred), 3))
print("Precision: ", round(precision_score(Y_test_lasso, y_pred, average="macro"), 3))
print("Recall: ",  round(recall_score(Y_test_lasso, y_pred, average="macro"), 3)) 
print("F1 Score: ", round(f1_score(Y_test_lasso, y_pred, average="macro"), 3)) 

Balanced accuracy:  0.686
Accuracy:  0.869
Precision:  0.875
Recall:  0.686
F1 Score:  0.715


In [ ]:
! pip install shap

In [ ]:
import shap
shap.initjs()

In [ ]:
# load JS visualization code to notebook
shap.initjs()

explainer = shap.Explainer(LogReg_trained, X_train_lasso, feature_names=X_train_lasso.columns[: 100])
# shap_values = explainer(X_test_lasso)
shap_values = explainer.shap_values(X_test_lasso)

shap.summary_plot(shap_values, features=X_train_lasso, feature_names=X_train_lasso.columns)
shap.summary_plot(shap_values, features=X_train_lasso, feature_names=X_train_lasso.columns, plot_type='bar')

In [ ]:
# EVALUATION with RandomForrest
from sklearn.ensemble import RandomForestClassifier

RF_trained = RandomForestClassifier().fit(X_train_lasso, Y_train_lasso)

y_pred=RF_trained.predict(X_test_lasso)
print("Balanced accuracy: ", round(balanced_accuracy_score(Y_test_lasso, y_pred), 3))
print("Accuracy: ", round(accuracy_score(Y_test_lasso, y_pred), 3))
print("Precision: ", round(precision_score(Y_test_lasso, y_pred, average="macro"), 3))
print("Recall: ",  round(recall_score(Y_test_lasso, y_pred, average="macro"), 3)) 
print("F1 Score: ", round(f1_score(Y_test_lasso, y_pred, average="macro"), 3)) 

Balanced accuracy:  0.689
Accuracy:  0.861
Precision:  0.881
Recall:  0.689
F1 Score:  0.722


In [ ]:
import shap
# load JS visualization code to notebook
shap.initjs()

explainer = shap.TreeExplainer(RF_trained, X_train_lasso, feature_names=X_train_lasso.columns)
# shap_values = explainer(X_test_lasso)
shap_values = explainer.shap_values(X_train_lasso)

shap.summary_plot(shap_values, features=X_train_lasso, feature_names=X_train_lasso.columns)
shap.summary_plot(shap_values, features=X_train_lasso, feature_names=X_train_lasso.columns, plot_type='bar')